In [0]:
import xml.etree.ElementTree as ET

In [0]:
%sh
pip install spark-xml

ERROR: Could not find a version that satisfies the requirement spark-xml (from versions: none)
ERROR: No matching distribution found for spark-xml
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-425920c6-12d2-4716-bab5-ea8af7b78da3/bin/python -m pip install --upgrade pip' command.


# Connection to blob

In [0]:
storageAccount = "datalakefabdeprojetos"
containerName = "ingrid-sollim"
mountpoint = "/mnt/ingrid-sollim/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope="Kv-fab-de-proj-dev",key="kv-ingrid-sollim")
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"
try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails


Mount point '/mnt/ingrid-sollim/' is already mounted.


In [0]:
path=mountpoint+"bikes"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
dbfs:/mnt/ingrid-sollim/bikes/customer.csv,customer.csv,1813963,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/person.csv,person.csv,13646947,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/product.csv,product.csv,104823,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderdetail.csv,salesorderdetail.csv,13801182,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderheader.csv,salesorderheader.csv,8267704,1694474862000
dbfs:/mnt/ingrid-sollim/bikes/spoffer.csv,spoffer.csv,36680,1694474860000


In [0]:
df = spark.read.option("header",True).options(delimiter=";").csv(path+"/salesorderheader.csv")
display(df.limit(10))

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
43659,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,NULL,"20565,6206","1971,5149","616,0984","23153,2339",NULL,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07 00:00:00.000
43660,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,NULL,"1294,2529","124,2483","38,8276","1457,3288",NULL,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07 00:00:00.000
43661,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,"32726,4786","3153,7696","985,553","36865,8012",NULL,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07 00:00:00.000
43662,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,"28832,5289","2775,1646","867,2389","32474,9324",NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07 00:00:00.000
43663,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,NULL,"419,4589","40,2681","12,5838","472,3108",NULL,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07 00:00:00.000
43664,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,NULL,"24432,6088","2344,9921","732,81","27510,4109",NULL,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07 00:00:00.000
43665,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,NULL,"14352,7713","1375,9427","429,9821","16158,6961",NULL,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07 00:00:00.000
43666,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,NULL,"5056,4896","486,3747","151,9921","5694,8564",NULL,E2A90057-1366-4487-8A7E-8085845FF770,2011-06-07 00:00:00.000
43667,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,NULL,"6107,082","586,1203","183,1626","6876,3649",NULL,86D5237D-432D-4B21-8ABC-671942F5789D,2011-06-07 00:00:00.000
43668,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,"35944,1562","3461,7654","1081,8017","40487,7233",NULL,281CC355-D538-494E-9B44-461B36A826C6,2011-06-07 00:00:00.000


In [0]:
df.printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- RevisionNumber: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- DueDate: string (nullable = true)
 |-- ShipDate: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- OnlineOrderFlag: string (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- PurchaseOrderNumber: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- SalesPersonID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- BillToAddressID: string (nullable = true)
 |-- ShipToAddressID: string (nullable = true)
 |-- ShipMethodID: string (nullable = true)
 |-- CreditCardID: string (nullable = true)
 |-- CreditCardApprovalCode: string (nullable = true)
 |-- CurrencyRateID: string (nullable = true)
 |-- SubTotal: string (nullable = true)
 |-- TaxAmt: string (nullable = true)
 |-- Freight: string (nullable = true)
 |-- TotalDue: string (null

In [0]:
display(df.select("ShipToAddressID").distinct())

ShipToAddressID
17401
829
25894
14369
27248
28503
13192
467
14157
16576


In [0]:
from pyspark.sql.functions import expr, col, regexp_replace,regexp_extract,when, to_date,to_timestamp

#Check for missing data

In [0]:
def replace_string_null(df, columns):
    for column_name in columns:
        df = df.withColumn(column_name, when(col(column_name) == "NULL", None).otherwise(col(column_name)))
    return df

# Example usage:
columns_to_replace = df.columns
null_replaced = replace_string_null(df, columns_to_replace)
display(null_replaced.limit(10))

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
43659,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,null,"20565,6206","1971,5149","616,0984","23153,2339",null,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07 00:00:00.000
43660,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,null,"1294,2529","124,2483","38,8276","1457,3288",null,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07 00:00:00.000
43661,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,"32726,4786","3153,7696","985,553","36865,8012",null,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07 00:00:00.000
43662,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,"28832,5289","2775,1646","867,2389","32474,9324",null,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07 00:00:00.000
43663,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,null,"419,4589","40,2681","12,5838","472,3108",null,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07 00:00:00.000
43664,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,null,"24432,6088","2344,9921","732,81","27510,4109",null,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07 00:00:00.000
43665,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,null,"14352,7713","1375,9427","429,9821","16158,6961",null,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07 00:00:00.000
43666,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,null,"5056,4896","486,3747","151,9921","5694,8564",null,E2A90057-1366-4487-8A7E-8085845FF770,2011-06-07 00:00:00.000
43667,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,null,"6107,082","586,1203","183,1626","6876,3649",null,86D5237D-432D-4B21-8ABC-671942F5789D,2011-06-07 00:00:00.000
43668,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,"35944,1562","3461,7654","1081,8017","40487,7233",null,281CC355-D538-494E-9B44-461B36A826C6,2011-06-07 00:00:00.000


In [0]:
display(null_replaced.select("OnlineOrderFlag").distinct())

OnlineOrderFlag
0
1


#Change Data types

In [0]:
null_replaced.printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- RevisionNumber: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- DueDate: string (nullable = true)
 |-- ShipDate: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- OnlineOrderFlag: string (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- PurchaseOrderNumber: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- SalesPersonID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- BillToAddressID: string (nullable = true)
 |-- ShipToAddressID: string (nullable = true)
 |-- ShipMethodID: string (nullable = true)
 |-- CreditCardID: string (nullable = true)
 |-- CreditCardApprovalCode: string (nullable = true)
 |-- CurrencyRateID: string (nullable = true)
 |-- SubTotal: string (nullable = true)
 |-- TaxAmt: string (nullable = true)
 |-- Freight: string (nullable = true)
 |-- TotalDue: string (null

In [0]:
#Replace comma for period
def replace_comma_with_period(df, columns_to_replace):
    for column in columns_to_replace:
        df = df.withColumn(column, regexp_replace(col(column), ",", "."))

    return df
#List of columns to replace
comma_replaced = ["SubTotal","TaxAmt","Freight","TotalDue"]

#replaced df
comma_replaced = replace_comma_with_period(null_replaced, columns_to_replace)
display(comma_replaced.limit(10))


SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
43659,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,null,20565.6206,1971.5149,616.0984,23153.2339,null,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07 00:00:00.000
43660,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,null,1294.2529,124.2483,38.8276,1457.3288,null,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07 00:00:00.000
43661,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,32726.4786,3153.7696,985.553,36865.8012,null,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07 00:00:00.000
43662,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,28832.5289,2775.1646,867.2389,32474.9324,null,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07 00:00:00.000
43663,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,null,419.4589,40.2681,12.5838,472.3108,null,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07 00:00:00.000
43664,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,null,24432.6088,2344.9921,732.81,27510.4109,null,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07 00:00:00.000
43665,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,null,14352.7713,1375.9427,429.9821,16158.6961,null,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07 00:00:00.000
43666,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,null,5056.4896,486.3747,151.9921,5694.8564,null,E2A90057-1366-4487-8A7E-8085845FF770,2011-06-07 00:00:00.000
43667,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,null,6107.082,586.1203,183.1626,6876.3649,null,86D5237D-432D-4B21-8ABC-671942F5789D,2011-06-07 00:00:00.000
43668,8,2011-05-31 00:00:00.000,2011-06-12 00:00:00.000,2011-06-07 00:00:00.000,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,35944.1562,3461.7654,1081.8017,40487.7233,null,281CC355-D538-494E-9B44-461B36A826C6,2011-06-07 00:00:00.000


In [0]:
#Change date
def change_date_format(df, columns_to_change):
    for column in columns_to_change:
        df = df.withColumn(column, to_date(to_timestamp(col(column), "yyyy-MM-dd HH:mm:ss.SSS"), "yyyy-MM-dd"))

    return df

#Change to int
def change_data_type_to_int(df, columns_to_change):
    for column in columns_to_change:
        df = df.withColumn(column, col(column).cast("int"))

    return df

#Change to float
def change_data_type_to_float(df, columns_to_change):
    for column in columns_to_change:
        df = df.withColumn(column, col(column).cast("float"))

    return df

In [0]:
# Define the list of columns to change date format
columns_to_change_date = ["OrderDate", "DueDate","ShipDate","ModifiedDate"]
result_date = change_date_format(comma_replaced, columns_to_change_date)

#Columns to change to int
columns_to_change_int = ["SalesOrderID","RevisionNumber","Status","OnlineOrderFlag","CustomerID","SalesPersonID",\
    "CustomerID","SalesPersonID","TerritoryID","BillToAddressID","CreditCardID","ShipMethodID","CurrencyRateID",\
       "CurrencyRateID","ShipToAddressID"]
result_int=change_data_type_to_int(result_date,columns_to_change_int)

#Columns to change to float
columns_to_change_float = ["SubTotal","TaxAmt","Freight","TotalDue"]
result_float = change_data_type_to_float(result_int,columns_to_change_float)

display(result_float.limit(10))

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
43659,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,null,20565.621,1971.5149,616.0984,23153.234,null,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07
43660,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,null,1294.2529,124.2483,38.8276,1457.3289,null,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
43661,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,32726.479,3153.7695,985.553,36865.8,null,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
43662,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,28832.53,2775.1646,867.2389,32474.932,null,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
43663,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,null,419.4589,40.2681,12.5838,472.3108,null,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
43664,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,null,24432.61,2344.9922,732.81,27510.41,null,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07
43665,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,null,14352.771,1375.9427,429.9821,16158.696,null,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07
43666,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,null,5056.4897,486.3747,151.9921,5694.8564,null,E2A90057-1366-4487-8A7E-8085845FF770,2011-06-07
43667,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,null,6107.082,586.1203,183.1626,6876.3647,null,86D5237D-432D-4B21-8ABC-671942F5789D,2011-06-07
43668,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,35944.156,3461.7654,1081.8018,40487.723,null,281CC355-D538-494E-9B44-461B36A826C6,2011-06-07


In [0]:
result_float.printSchema()

root
 |-- SalesOrderID: integer (nullable = true)
 |-- RevisionNumber: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- DueDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- Status: integer (nullable = true)
 |-- OnlineOrderFlag: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- PurchaseOrderNumber: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- SalesPersonID: integer (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- BillToAddressID: integer (nullable = true)
 |-- ShipToAddressID: integer (nullable = true)
 |-- ShipMethodID: integer (nullable = true)
 |-- CreditCardID: integer (nullable = true)
 |-- CreditCardApprovalCode: string (nullable = true)
 |-- CurrencyRateID: integer (nullable = true)
 |-- SubTotal: float (nullable = true)
 |-- TaxAmt: float (nullable = true)
 |-- Freight: float (nullable = true)
 |-- TotalDue: float (nu

In [0]:
result_float.columns

Out[33]: ['SalesOrderID',
 'RevisionNumber',
 'OrderDate',
 'DueDate',
 'ShipDate',
 'Status',
 'OnlineOrderFlag',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'SalesPersonID',
 'TerritoryID',
 'BillToAddressID',
 'ShipToAddressID',
 'ShipMethodID',
 'CreditCardID',
 'CreditCardApprovalCode',
 'CurrencyRateID',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'Comment',
 'rowguid',
 'ModifiedDate']

In [0]:
column_mapping = {
'SalesOrderID':'sales_order_id',
 'RevisionNumber':'revision_number',
 'OrderDate':'order_date',
 'DueDate':'due_date',
 'ShipDate':'ship_date',
 'Status':'status',
 'OnlineOrderFlag':'online_order_flag',
 'SalesOrderNumber':'sales_order_number',
 'PurchaseOrderNumber':'purchase_order_number',
 'AccountNumber':'account_number',
 'CustomerID':'customer_id',
 'SalesPersonID':'sales_person_id',
 'TerritoryID':'territory_id',
 'BillToAddressID':'billTo_address_id',
 'ShipToAddressID':'ship_to_address_id',
 'ShipMethodID':'ship_method_id',
 'CreditCardID':'credit_card_id',
 'CreditCardApprovalCode':'credit_card_approval_code',
 'CurrencyRateID':'currency_rate_id',
 'SubTotal':'sub_total',
 'TaxAmt':'tax_amt',
 'Freight':'freight',
 'TotalDue':'total_due',
 'Comment':'comment',
 'rowguid':'rowguid',
 'ModifiedDate':'modified_date'
}

# Create a list of column expressions with aliases
select_expr = [col(old_col).alias(new_col) for old_col, new_col in column_mapping.items()]

# Apply the column renaming using select and alias
sales_header = result_float.select(*select_expr)
display(sales_header.limit(10))

sales_order_id,revision_number,order_date,due_date,ship_date,status,online_order_flag,sales_order_number,purchase_order_number,account_number,customer_id,sales_person_id,territory_id,billTo_address_id,ship_to_address_id,ship_method_id,credit_card_id,credit_card_approval_code,currency_rate_id,sub_total,tax_amt,freight,total_due,comment,rowguid,modified_date
43659,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,null,20565.621,1971.5149,616.0984,23153.234,null,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07
43660,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,null,1294.2529,124.2483,38.8276,1457.3289,null,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
43661,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,32726.479,3153.7695,985.553,36865.8,null,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
43662,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,28832.53,2775.1646,867.2389,32474.932,null,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
43663,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,null,419.4589,40.2681,12.5838,472.3108,null,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
43664,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,null,24432.61,2344.9922,732.81,27510.41,null,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07
43665,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,null,14352.771,1375.9427,429.9821,16158.696,null,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07
43666,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,null,5056.4897,486.3747,151.9921,5694.8564,null,E2A90057-1366-4487-8A7E-8085845FF770,2011-06-07
43667,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,null,6107.082,586.1203,183.1626,6876.3647,null,86D5237D-432D-4B21-8ABC-671942F5789D,2011-06-07
43668,8,2011-05-31,2011-06-12,2011-06-07,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,35944.156,3461.7654,1081.8018,40487.723,null,281CC355-D538-494E-9B44-461B36A826C6,2011-06-07


In [0]:
sales_header.printSchema()

root
 |-- sales_order_id: integer (nullable = true)
 |-- revision_number: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- due_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- status: integer (nullable = true)
 |-- online_order_flag: integer (nullable = true)
 |-- sales_order_number: string (nullable = true)
 |-- purchase_order_number: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- sales_person_id: integer (nullable = true)
 |-- territory_id: integer (nullable = true)
 |-- billTo_address_id: integer (nullable = true)
 |-- ship_to_address_id: integer (nullable = true)
 |-- ship_method_id: integer (nullable = true)
 |-- credit_card_id: integer (nullable = true)
 |-- credit_card_approval_code: string (nullable = true)
 |-- currency_rate_id: integer (nullable = true)
 |-- sub_total: float (nullable = true)
 |-- tax_amt: float (nullable = true)
 |-- freight: float (nullable

In [0]:
sales_header.columns

Out[25]: ['sales_order_id',
 'revision_number',
 'order_date',
 'due_date',
 'ship_date',
 'status',
 'online_order_flag',
 'sales_order_number',
 'purchase_order_number',
 'account_number',
 'customer_id',
 'sales_person_id',
 'territory_id',
 'billTo_address_id',
 'ship_to_address_id',
 'ship_method_id',
 'credit_card_id',
 'credit_card_approval_code',
 'currency_rate_id',
 'sub_total',
 'tax_amt',
 'freight',
 'total_due',
 'comment',
 'rowguid',
 'modified_date']

#Ingest the cleaned data to the database

### Connection to the database

In [0]:
# Get the secrets from the jdbc secret scope.
username = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dblogin")
password = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dbpass")

jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "db-fab-projetos"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.sales_header'

### Insert into the database


In [0]:
sales_header.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()

In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[30]: True